In [1]:
from datetime import datetime
res = !gcloud config get core/project
PROJECT_ID = res[0]
CONTAINER_IMAGE_NAME="impressions"
GCP_REGION='us-central1'
TAG_NAME = 'masterv9'

# KAGGLE_KEY=''
CONTAINER_IMAGE_NAME_DATA_PREP = f"{CONTAINER_IMAGE_NAME}-data-preparation"
CONTAINER_IMAGE_NAME_FINE_TUNE = f"{CONTAINER_IMAGE_NAME}-trainning"
CONTAINER_IMAGE_NAME_RUN_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"

In [2]:
import os
print(os.getcwd())

/home/andre/impressions


In [3]:
from components.data import youtube_captions, prepare_text_data
from components.utils import gcs_utils
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
url =  'https://www.youtube.com/watch?v=OuF9weSkS68&list=PL0iVR8sl9TiWqj_JmVjEgAzl4YhtTt9Wf' #'https://www.youtube.com/watch?v=pCX_3p40Efc'
id_type = 'playlist'
language = 'en'
blob_name = 'ColdFusion' # 'SentdexChannel'
bucket_name = 'metal-sky-419309-videos-v1'
development_percentage = 1
train_percentage = .9
tokenizer_type = 'gpt-4'
max_number_videos = 20

config = dict(bucket_name=bucket_name, url=url, id_type=id_type, language=language, blob_name=blob_name,
              max_number_videos=max_number_videos, development_percentage=development_percentage, train_percentage=train_percentage,
              tokenizer_type=tokenizer_type)
for a, b in config.items():
    print(a, b)

bucket_name metal-sky-419309-videos-v1
url https://www.youtube.com/watch?v=OuF9weSkS68&list=PL0iVR8sl9TiWqj_JmVjEgAzl4YhtTt9Wf
id_type playlist
language en
blob_name ColdFusion
max_number_videos 20
development_percentage 1
train_percentage 0.9
tokenizer_type gpt-4


In [5]:
# videos = youtube_captions.get_videos_and_captions(config['url'], config['id_type'], config['language'])
# if videos:
#     gcs_utils.save_dict_to_gcs(videos, bucket_name=config['bucket_name'], blob_name = config['blob_name'])
# else:
#     logger.error("No video data to save.")

In [6]:
from components.config.Config import ConfigGPT
config = ConfigGPT()
config.dataset = blob_name
print(config)

Simon
Configuration:
out_dir: out
eval_interval: 200
log_interval: 1
eval_iters: 200
eval_only: False
always_save_checkpoint: True
init_from: scratch
dataset: ColdFusion
block_size: 32
batch_size: 8
n_layer: 4
n_head: 4
n_embd: 12
dropout: 0.0
bias: False
learning_rate: 0.0006
max_iters: 400
decay_lr: True
min_lr: 6e-05
grad_clip: 1.0
seed_offset: 0
device: cpu
dtype: float16
compile: True
model_name: Impressions



In [7]:

prepare_text_data.process_captions_and_prepare_data(bucket_name, blob_name, development_percentage, train_percentage, tokenizer_type, language)

Blob ColdFusion read successfully.
Video c49WRDVCvpo does not have captions in Spanish.
Video Cn4pNfudPHc does not have captions in Spanish.
Video 2qMl8JkJ5hg does not have captions in Spanish.
Video 3DUb1P5HjlQ does not have captions in Spanish.
Master caption length: 949675
The tokenizer type is : gpt-4
This is the known token idx : [1, 4, 6, 8, 11]
This is the unknown token idx : [1, 4, 6, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 65555, 32795, 65564, 32797, 30, 33, 32802, 35, 36, 37, 40, 42, 45, 32813, 47, 50, 51, 52, 53, 54, 55, 56, 57, 60, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 65606, 74, 76, 77, 78, 98383, 79, 81, 82, 32847, 84, 85, 98386, 80, 88, 98392, 65623, 86, 32860, 32855, 87, 89, 65632, 32874, 98417, 98421, 32885, 65655, 32888, 32889, 32887, 98431, 32905, 98444, 98445, 65685, 65691, 98461, 65695, 32931, 98475, 32942, 65712, 65714, 32946, 32947, 32951, 65726, 32959, 32961, 98504, 32980, 32981, 220, 65761, 65765, 65766, 65767, 65770, 98538, 98541, 3300

In [8]:
import torch
from components.training.train import Trainer

print(config)
print("Training...")
trainer = Trainer(config)
model = trainer.train()


Simon
Configuration:
out_dir: out
eval_interval: 200
log_interval: 1
eval_iters: 200
eval_only: False
always_save_checkpoint: True
init_from: scratch
dataset: ColdFusion
block_size: 32
batch_size: 8
n_layer: 4
n_head: 4
n_embd: 12
dropout: 0.0
bias: False
learning_rate: 0.0006
max_iters: 400
decay_lr: True
min_lr: 6e-05
grad_clip: 1.0
seed_offset: 0
device: cpu
dtype: float16
compile: True
model_name: Impressions

Training...
loading meta data from components/data/ColdFusion/meta.pkl
found vocab_size = 12529 (inside components/data/ColdFusion/meta.pkl)
number of parameters: 0.16M


/opt/conda/lib/python3.10/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


compiling the model... (takes a ~minute)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


RuntimeError: index out of bounds: 0 <= tmp3 < 12529L
Exception raised from kernel at /var/tmp/torchinductor_andre/2u/c2uvifeccisyrob2nue6ebkte477rgwr7sssftagl2gnvbinr5jf.cpp:40 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x7fd94c12f897 in /opt/conda/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, char const*) + 0x68 (0x7fd94c0dfbee in /opt/conda/lib/python3.10/site-packages/torch/lib/libc10.so)
frame #2: kernel + 0x422 (0x7fd94c0102b2 in /var/tmp/torchinductor_andre/2u/c2uvifeccisyrob2nue6ebkte477rgwr7sssftagl2gnvbinr5jf.so)
frame #3: <unknown function> + 0x340b (0x7fd94c01040b in /var/tmp/torchinductor_andre/2u/c2uvifeccisyrob2nue6ebkte477rgwr7sssftagl2gnvbinr5jf.so)
frame #4: <unknown function> + 0x1445c8 (0x564b0ed9b5c8 in /opt/conda/bin/python)
frame #5: _PyObject_MakeTpCall + 0x26b (0x564b0ed94a6b in /opt/conda/bin/python)
frame #6: _PyEval_EvalFrame

In [ ]:
from components.data import tokenizer
tokenizer_instance = tokenizer.TextTokenizer(tokenizer_type=trainer.meta['tokenizer_type'], stoi=trainer.stoi, itos=trainer.itos)

initial_text = "* Ay wey"
encoded_input = tokenizer_instance.encode(initial_text)
context = torch.tensor(encoded_input, dtype=torch.long, device=config.device)
context = context.unsqueeze(0)
generated_encoded_text = model.generate(context, max_new_tokens=50)[0].tolist()
print(tokenizer_instance.decode(generated_encoded_text))

In [ ]:
encoded_input

In [ ]:
# import argparse
# from components.config.Config import ConfigGPT, override_config_with_args

# def parse_args():
#     parser = argparse.ArgumentParser(description="Training Configuration Overrides")

#     # General settings
#     parser.add_argument('--random_seed', type=int, help='Random seed for reproducibility')
#     parser.add_argument('--device', type=str, choices=['cuda', 'cpu'], help='Device to run the training on')

#     # Data settings
#     parser.add_argument('--dataset_path', type=str, help='Path to the dataset')
#     parser.add_argument('--batch_size', type=int, help='Batch size for training')
#     parser.add_argument('--num_workers', type=int, help='Number of data loading workers')

#     # Model settings
#     parser.add_argument('--model_name', type=str, help='Model architecture to use')
#     parser.add_argument('--pretrained', type=bool, help='Use pretrained model weights')
#     parser.add_argument('--num_classes', type=int, help='Number of output classes')

#     # Training settings
#     parser.add_argument('--learning_rate', type=float, help='Learning rate for the optimizer')
#     parser.add_argument('--epochs', type=int, help='Number of training epochs')
#     parser.add_argument('--weight_decay', type=float, help='Weight decay (L2 regularization)')

#     return parser.parse_args()

# def main():
#     # Load default configuration
#     config = ConfigGPT()
#     print(config)

#     # Parse command-line arguments
#     args = parse_args()

#     # Override configuration with command-line arguments
#     override_config_with_args(config, args)

#     # Display the final configuration
#     print(config)

#     # Here, you would normally continue with the rest of your training script
#     # For example:
#     # train_model(config)

# if __name__ == "__main__":
#     main()